# Workflow para generar submit con predicciones generadas por metamodelos

## Inicializacion

In [108]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 18:59:59 -03"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1537227,82.1,2509441,134.1,2509441,134.1
Vcells,2949571,22.6,1300162378,9919.5,1316026612,10040.5


In [ ]:
PARAM <- list()
PARAM$experimento <- "wfzext9219_real"
PARAM$semilla_primigenia <- 999199

In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )

#dataset <- fread("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-16 18:59:59 -03"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1538271,82.2,2509441,134.1,2509441,134.1
Vcells,723614254,5520.8,1300162378,9919.5,1316026612,10040.5


[1] "2025-11-16 19:00:44 -03"

In [112]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


In [113]:
ncol(dataset)
colnames(dataset)

[1] 155

[1] "numero_de_cliente"                   
  [2] "foto_mes"                            
  [3] "active_quarter"                      
  [4] "cliente_vip"                         
  [5] "internet"                            
  [6] "cliente_edad"                        
  [7] "cliente_antiguedad"                  
  [8] "mrentabilidad"                       
  [9] "mrentabilidad_annual"                
 [10] "mcomisiones"                         
 [11] "mactivos_margen"                     
 [12] "mpasivos_margen"                     
 [13] "cproductos"                          
 [14] "tcuentas"                            
 [15] "ccuenta_corriente"                   
 [16] "mcuenta_corriente_adicional"         
 [17] "mcuenta_corriente"                   
 [18] "ccaja_ahorro"                        
 [19] "mcaja_ahorro"                        
 [20] "mcaja_ahorro_adicional"              
 [21] "mcaja_ahorro_dolares"                
 [22] "cdescubierto_preacordado"            
 [23] "mcuentas_saldo"                      
 [24] "ctarjeta_debito"                     
 [25] "ctarjeta_debito_transacciones"       
 [26] "mautoservicio"                       
 [27] "ctarjeta_visa"                       
 [28] "ctarjeta_visa_transacciones"         
 [29] "mtarjeta_visa_consumo"               
 [30] "ctarjeta_master"                     
 [31] "ctarjeta_master_transacciones"       
 [32] "mtarjeta_master_consumo"             
 [33] "cprestamos_personales"               
 [34] "mprestamos_personales"               
 [35] "cprestamos_prendarios"               
 [36] "mprestamos_prendarios"               
 [37] "cprestamos_hipotecarios"             
 [38] "mprestamos_hipotecarios"             
 [39] "cplazo_fijo"                         
 [40] "mplazo_fijo_dolares"                 
 [41] "mplazo_fijo_pesos"                   
 [42] "cinversion1"                         
 [43] "minversion1_pesos"                   
 [44] "minversion1_dolares"                 
 [45] "cinversion2"                         
 [46] "minversion2"                         
 [47] "cseguro_vida"                        
 [48] "cseguro_auto"                        
 [49] "cseguro_vivienda"                    
 [50] "cseguro_accidentes_personales"       
 [51] "ccaja_seguridad"                     
 [52] "cpayroll_trx"                        
 [53] "mpayroll"                            
 [54] "mpayroll2"                           
 [55] "cpayroll2_trx"                       
 [56] "ccuenta_debitos_automaticos"         
 [57] "mcuenta_debitos_automaticos"         
 [58] "ctarjeta_visa_debitos_automaticos"   
 [59] "mttarjeta_visa_debitos_automaticos"  
 [60] "ctarjeta_master_debitos_automaticos" 
 [61] "mttarjeta_master_debitos_automaticos"
 [62] "cpagodeservicios"                    
 [63] "mpagodeservicios"                    
 [64] "cpagomiscuentas"                     
 [65] "mpagomiscuentas"                     
 [66] "ccajeros_propios_descuentos"         
 [67] "mcajeros_propios_descuentos"         
 [68] "ctarjeta_visa_descuentos"            
 [69] "mtarjeta_visa_descuentos"            
 [70] "ctarjeta_master_descuentos"          
 [71] "mtarjeta_master_descuentos"          
 [72] "ccomisiones_mantenimiento"           
 [73] "mcomisiones_mantenimiento"           
 [74] "ccomisiones_otras"                   
 [75] "mcomisiones_otras"                   
 [76] "cforex"                              
 [77] "cforex_buy"                          
 [78] "mforex_buy"                          
 [79] "cforex_sell"                         
 [80] "mforex_sell"                         
 [81] "ctransferencias_recibidas"           
 [82] "mtransferencias_recibidas"           
 [83] "ctransferencias_emitidas"            
 [84] "mtransferencias_emitidas"            
 [85] "cextraccion_autoservicio"            
 [86] "mextraccion_autoservicio"            
 [87] "ccheques_depositados"                
 [88] "mcheques_depositados"                
 [89] "ccheques_emitidos"                 

### Final Training Strategy

In [114]:
PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912
, 202001
, 202002, 202003 
, 202004, 202005, 202006, 202007, 202008, 202009
, 202010, 202011, 202012
, 202101, 202102, 202103, 202104
, 202105, 202106
)

PARAM$train_final$undersampling <- 0.1 

In [115]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [116]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [117]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [ ]:
campos_buenos <- setdiff(colnames(dataset_train_final), c("clase_ternaria","clase01","training"))



### funciones test

In [119]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

PARAM$semilla_kaggle <- 314159

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle 
  )

  return( drealidad )
}

# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}


### Predicciones externas

In [ ]:
PARAM$false_future <- c(202108) 
d_false_future <- dataset[foto_mes %in% PARAM$false_future]

# FUTURO REAL. Predicciones de los modelos entrenados con ganancia en test 202106 > 416 millones [suman 600 semillas entre los 4 modelos]
urls_false_future <- c(
 "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9306/prediccion_future_real_wfzr9306.txt",
 "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9306_bis/prediccion_future_real_wfzr9306_bis.txt",
 "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9321/prediccion_future_real_wfzr9321.txt",
 "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9321bis/prediccion_future_real_wfzr9321bis.txt"
)

# Conjunto predicciones de los modelos entrenados en futuro real. promedio ganancia en test 202106 > 405 millones [suman  1600 semillas entre los 20 modelos]
# urls_false_future <- c(
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9201/prediccion_future_wfzr9201.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9205/prediccion_future_wfzr9205.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9106/prediccion_future_wfzr9106.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9208/prediccion_future_wfzr9208.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9210/prediccion_future_wfzr9210.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9213/prediccion_future_wfzr9213.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9118/prediccion_future_wfzr9118.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9112/prediccion_future_wfzr9112.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9203/prediccion_future_wfzr9203.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9212/prediccion_future_wfzr9212.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9217/prediccion_future_wfzr9217.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9304/prediccion_future_real_wfzr9304.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9307/prediccion_future_real_wfzr9307.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9310/prediccion_future_real_wfzr9310.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9313/prediccion_future_real_wfzr9313.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9315/prediccion_future_real_wfzr9315.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9318/prediccion_future_real_wfzr9318.txt",
#   "https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9319/prediccion_future_real_wfzr9319.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9321/prediccion_future_real_wfzr9321.txt",
#   "https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9321bis/prediccion_future_real_wfzr9321bis.txt"
# )

tb_prediccion_false_future_prom <- fread(urls_false_future[1], sep = "\t")
setorder(tb_prediccion_false_future_prom, numero_de_cliente, foto_mes)

vpred_acum_false <- tb_prediccion_false_future_prom$prob
qacumulados_false <- 1L

if (length(urls_false_future) > 1L) {
  for (url_i in urls_false_future[-1]) {
    dt <- fread(url_i, sep = "\t")
    setorder(dt, numero_de_cliente, foto_mes)

    if (!all(dt$numero_de_cliente == tb_prediccion_false_future_prom$numero_de_cliente) ||
        !all(dt$foto_mes == tb_prediccion_false_future_prom$foto_mes)) {
      stop(sprintf("Las llaves no coinciden en %s", url_i))
    }

    vpred_acum_false <- vpred_acum_false + dt$prob
    qacumulados_false <- qacumulados_false + 1L

    cat(" -> Leída y sumada:", url_i, "\n")
  }
}
vpred_acum_false <- vpred_acum_false / qacumulados_false

tb_prediccion_false_future_prom[, prob := vpred_acum_false]

fwrite(
  tb_prediccion_false_future_prom[, .(numero_de_cliente, foto_mes, prob)],
  file = paste0("prediccion_ext_future_ensemble_", PARAM$experimento, ".txt"),
  sep = "\t"
)

cat("\nPredicción promedio FINAL guardada en: prediccion_ext_future_ensemble_", PARAM$experimento, ".txt\n")




 -> Leída y sumada: https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9205/prediccion_future_wfzr9205.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9106/prediccion_future_wfzr9106.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9208/prediccion_future_wfzr9208.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9210/prediccion_future_wfzr9210.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9213/prediccion_future_wfzr9213.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzr9118/prediccion_future_wfzr9118.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9112/prediccion_future_wfzr9112.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzr9203/prediccion_future_wfzr9203.txt 
 -> Leída y sumada:

### submit at 11000
se eligió analizando el mejor corte entre los 30 experimentos que más ganancia dieron en testing

In [137]:
### kaggle 

dir.create("kaggle_real", showWarnings = FALSE)

PARAM$cortes <- 11000

tb_ord_test <- copy(tb_prediccion_false_future_prom)
setorder(tb_ord_test, -prob)

for (envios in PARAM$cortes) {
  tb_ord_test[, Predicted := 0L]                          
  tb_ord_test[1:min(envios, .N), Predicted := 1L]        

  archivo_kaggle_real <- paste0("./kaggle_real/KA_real", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_ord_test[, .(numero_de_cliente, Predicted)],
        file = archivo_kaggle_real,
        sep = ",",
        col.names = TRUE)

  cat("Generado:", basename(archivo_kaggle_real), "\n")
}


Generado: KA_realwfzext9219_11000.csv 


In [138]:
# --- Bloque de generación y control de archivo _format ---

archivo_kaggle_format <- sub("\\.csv$", "_format\\.csv", archivo_kaggle_real) 

# 3. CARGAR EL ARCHIVO GENERADO
tb_kaggle <- fread(archivo_kaggle_real)

# 4. FILTRAR los registros donde Predicted es 1
tb_format <- tb_kaggle[Predicted == 1]

# 5. IMPLEMENTAR EL CONTROL de 11,000 filas
filas_finales <- nrow(tb_format)
meta_filas <- 11000

if (filas_finales == meta_filas) {
  message(paste(" Control exitoso. Se encontraron", filas_finales, "filas con Predicted = 1."))
  
  fwrite(
    # solo la columna 'numero_de_cliente'
    tb_format[, .(numero_de_cliente)], 
    file = archivo_kaggle_format,
    sep = ",",
    # col.names = FALSE: ELIMINA el encabezado (primera fila)
    col.names = FALSE, 
    quote = FALSE 
  )
  cat("Generado formato:", basename(archivo_kaggle_format), "\n")
  
} else {
  stop(paste(" ERROR: El archivo final debe tener", meta_filas, "filas. Se encontraron", filas_finales, "filas. Revisar la predicción."))
}

 Control exitoso. Se encontraron 11000 filas con Predicted = 1.



Generado formato: KA_realwfzext9219_11000_format.csv 


In [ ]:
# # GANANCIAS  EN TEST  

# drealidad <- realidad_inicializar(d_false_future, PARAM) 

# ganancias_completas <- data.table()
# PARAM$cortes <- seq(7000, 16000, by = 500)


# cat("✓ Calculando ganancias para el Ensemble promedio...\n")
# setorder(tb_prediccion_false_future_prom, -prob)
# tb_ensemble <- copy(tb_prediccion_false_future_prom)
# mejor_ganancia_ensemble <- -Inf
# mejor_corte <- NA

# for (corte in PARAM$cortes) {
#   tb_ensemble[, Predicted := 0L]
#   tb_ensemble[1:corte, Predicted := 1L]

#   res <- realidad_evaluar(drealidad, tb_ensemble)

#   # Guardar fila del ensemble por corte
#   ganancias_completas <- rbind(
#     ganancias_completas,
#     data.table(
#       tipo = "Ensemble",
#       corte = corte,
#       ganancia = res$total
#     )
#   )

#   cat("Envios=", corte, "\t",
#       " TOTAL=",   format(res$total,   big.mark = ","),
#       " Public=",  format(res$public,  big.mark = ","),
#       " Private=", format(res$private, big.mark = ","),
#       "\n", sep = "")

#   if (res$total > mejor_ganancia_ensemble) {
#     mejor_ganancia_ensemble <- res$total
#     mejor_corte <- corte
#   }
# }
# cat("Corte óptimo del Ensemble (para línea roja):", mejor_corte, "envíos\n")

# archivo_completo <- paste0("ganancias_todas_semillas_cortes_", PARAM$experimento, ".txt")
# fwrite(ganancias_completas, file = archivo_completo, sep = "\t")
# cat("\n Ganancias completas (por corte) guardadas en:", archivo_completo, "\n")

# resultados_ganancia_ensemble <- ganancias_completas[
#   tipo == "Ensemble",
#   .(envios = corte, ganancia_total = ganancia)
# ][order(envios)]

# nombre_archivo_ganancias_ensemble <- paste0("ganancias_ensemble_exp_", PARAM$experimento, ".txt")
# fwrite(resultados_ganancia_ensemble, file = nombre_archivo_ganancias_ensemble, sep = "\t")
# cat(" Ganancias del Ensemble (derivadas) guardadas en:", nombre_archivo_ganancias_ensemble, "\n")



### grafico curva ganancia acumulada por cliente

In [126]:
# # Ordenar por probabilidad descendente
# setorder(tb_prediccion_false_future_prom, -prob)

# # Calcular ganancia acumulada
# tb_prediccion_false_future_prom[, indice := 1:.N]

# # Unir con drealidad para tener clase_ternaria
# tb_ganancia <- merge(
#   tb_prediccion_false_future_prom[, .(numero_de_cliente, foto_mes, prob, indice)],
#   drealidad[, .(numero_de_cliente, foto_mes, clase_ternaria)],
#   by = c("numero_de_cliente", "foto_mes")
# )

# # Ordenar por indice para mantener el orden
# setorder(tb_ganancia, indice)

# # Calcular ganancia individual y acumulada
# tb_ganancia[, ganancia_individual := ifelse(clase_ternaria == "BAJA+2", 780000, -20000)]
# tb_ganancia[, ganancia_acumulada := cumsum(ganancia_individual)]

# # Encontrar ganancia máxima
# ganancia_maxima <- max(tb_ganancia$ganancia_acumulada)
# indice_maximo <- tb_ganancia[ganancia_acumulada == ganancia_maxima, indice][1]

# # Filtrar datos (umbral 66% de ganancia máxima)
# umbral_ganancia <- ganancia_maxima * 0.66
# tb_filtrada <- tb_ganancia[ganancia_acumulada >= umbral_ganancia]

# # Crear gráfico
# p_acumulada <- ggplot(tb_filtrada, aes(x = indice, y = ganancia_acumulada)) +
#   geom_line(color = "blue", linewidth = 1.2) +
#   geom_point(data = tb_ganancia[indice == indice_maximo], 
#              aes(x = indice, y = ganancia_acumulada),
#              color = "red", size = 3) +
#   annotate("text", 
#            x = indice_maximo, 
#            y = ganancia_maxima * 1.05,
#            label = paste0("Ganancia Máxima\n", format(ganancia_maxima, big.mark = ",", scientific = FALSE)),
#            color = "red", 
#            fontface = "bold") +
#   annotate("segment",
#            x = indice_maximo, y = ganancia_maxima,
#            xend = indice_maximo, yend = ganancia_maxima * 1.04,
#            arrow = arrow(length = unit(0.3, "cm")),
#            color = "red") +
#   scale_x_continuous(labels = scales::comma) +
#   scale_y_continuous(labels = scales::comma) +
#   labs(
#     title = paste0("Ganancia acumulada por orden de predicción (filtrada) - EXP ", PARAM$experimento),
#     x = "Clientes ordenados por probabilidad",
#     y = "Ganancia Acumulada"
#   ) +
#   theme_minimal() +
#   theme(
#     plot.title = element_text(face = "bold", size = 14),
#     panel.grid.minor = element_line(linewidth = 0.3, linetype = "dotted")
#   )

# print(p_acumulada)

# nombre_archivo_acum <- paste0("curva_ganancia_acumulada_exp_", PARAM$experimento, ".png")
# ggsave(nombre_archivo_acum, p_acumulada, width = 14, height = 8, dpi = 300)

# cat("Ganancia máxima:", format(ganancia_maxima, big.mark = ","), "\n")
# cat("Corte ideal por cliente:", indice_maximo, "\n")